In [205]:
import warnings
warnings.filterwarnings("ignore")

In [206]:
!pip install webdriver-manager



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [207]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from time import sleep

from dotenv import load_dotenv
load_dotenv()

True

In [208]:
# if you don't have a .env file, you can create one and add the following lines:
os.environ['EMAIL'] = 'daziaconsulting@gmail.com'
os.environ['PASSWORD'] = 'Dazia@2024'   

In [209]:
driver = webdriver.Chrome()

In [210]:
driver.get('https://www.linkedin.com/login')

In [211]:
driver.title

'LinkedIn Login, Sign in | LinkedIn'

In [212]:
email = driver.find_element(By.ID, 'username')
email.send_keys(os.environ['EMAIL'])

password = driver.find_element(By.ID, 'password')
password.send_keys(os.environ['PASSWORD'])

password.submit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.267)
Stacktrace:
	GetHandleVerifier [0x00007FF6305080D5+2992373]
	(No symbol) [0x00007FF63019BFD0]
	(No symbol) [0x00007FF63003590A]
	(No symbol) [0x00007FF63000F4F5]
	(No symbol) [0x00007FF6300B63A7]
	(No symbol) [0x00007FF6300CEE72]
	(No symbol) [0x00007FF6300AF113]
	(No symbol) [0x00007FF63007A918]
	(No symbol) [0x00007FF63007BA81]
	GetHandleVerifier [0x00007FF630566A2D+3379789]
	GetHandleVerifier [0x00007FF63057C32D+3468109]
	GetHandleVerifier [0x00007FF630570043+3418211]
	GetHandleVerifier [0x00007FF6302FC78B+847787]
	(No symbol) [0x00007FF6301A757F]
	(No symbol) [0x00007FF6301A2FC4]
	(No symbol) [0x00007FF6301A315D]
	(No symbol) [0x00007FF630192979]
	BaseThreadInitThunk [0x00007FF99DF6E8D7+23]
	RtlUserThreadStart [0x00007FF99EFDFBCC+44]


In [190]:
## MAKE SURE TO USE ONLY THIS URL TO AVOID BEING STUCK IN ERRORS

url = "https://www.linkedin.com/in/hannah-nardone-126655282/"
driver.get(url)

In [191]:
profile_data = {}

In [ ]:
driver.title

'(21) Hannah Nardone | LinkedIn'

In [193]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

name = soup.find('h1')

name = name.get_text().strip()

profile_data['name'] = name
profile_data['url'] = url

In [194]:
profile_data

{'name': 'Hannah Nardone',
 'url': 'https://www.linkedin.com/in/hannah-nardone-126655282/'}

In [195]:
headline = soup.find('div', {'class': 'text-body-medium break-words'})
headline = headline.get_text().strip()

profile_data['headline'] = headline

In [196]:
profile_data

{'name': 'Hannah Nardone',
 'url': 'https://www.linkedin.com/in/hannah-nardone-126655282/',
 'headline': 'Mathematics Student at McMaster University'}

In [197]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

about = soup.find('div', {'class': 'display-flex ph5 pv3'})

about = about.get_text().strip()

profile_data['about'] = about

In [198]:
profile_data

{'name': 'Hannah Nardone',
 'url': 'https://www.linkedin.com/in/hannah-nardone-126655282/',
 'headline': 'Mathematics Student at McMaster University',
 'about': 'Hi! I’m Hannah, I am currently a third year student at McMaster University specializing in mathematics. I am passionate about pure math and eager to pursue research in this field! Hi! I’m Hannah, I am currently a third year student at McMaster University specializing in mathematics. I am passionate about pure math and eager to pursue research in this field!'}

In [199]:
url2 = f"{url}/overlay/contact-info/"
driver.get(url2)

In [200]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')


email = soup.find('div', {'class': 'sTcdifezBUxpROXbnKHLDEcFBCqbbiGT link-without-visited-state t-14'})

if email:
    email = email.get_text().strip()
else:
    print("blank")

profile_data['email'] = email

blank


In [201]:
profile_data

{'name': 'Hannah Nardone',
 'url': 'https://www.linkedin.com/in/hannah-nardone-126655282/',
 'headline': 'Mathematics Student at McMaster University',
 'about': 'Hi! I’m Hannah, I am currently a third year student at McMaster University specializing in mathematics. I am passionate about pure math and eager to pursue research in this field! Hi! I’m Hannah, I am currently a third year student at McMaster University specializing in mathematics. I am passionate about pure math and eager to pursue research in this field!',
 'email': None}

In [230]:
import os
from groq import Groq

# Initialize Groq API
os.environ["GROQ_API_KEY"] = ""
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def extract_dynamic_tags_and_summarize(input_data):
    """
    Extract dynamic tags based on the understanding of the AI model
    and summarize the 'about' section in 50 words.
    """
    try:
        # Extract headline and about section
        headline = input_data.get("headline", "")
        about = input_data.get("about", "")

        # AI prompt to extract dynamic tags
        tag_prompt = f"""
        Analyze the following headline and about section to extract key tags that best describe the person's background, roles, interests, and areas of expertise. Only in Tags format like McMaster University, Mathematics, Student etc.
        
        Headline: {headline}
        About: {about}
        
        Extract the tags dynamically and provide them as a python list of strings.
        """
        tag_response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are an expert AI for analyzing text and extracting meaningful tags."},
                {"role": "user", "content": tag_prompt}
            ]
        )
        tags = tag_response.choices[0].message.content.strip()
# tag_response.choices[0].message.content.strip()
        # AI prompt to summarize the 'about' section
        summary_prompt = f"""
        Summarize the following text into 50 words:
        "{about}"
        """
        summary_response = client.chat.completions.create(
            model="llama3-8b-8192",

            messages=[
                {"role": "system", "content": "You are a text summarization assistant."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        summary = summary_response.choices[0].message.content.strip()
# summary_response.choices[0].message.content.strip()
        return {
            "tags": tags,
            "summary": summary
        }

    except Exception as e:
        return {"error": str(e)}

# Input data
input_data = {
    'name': 'Hannah Nardone',
    'url': 'https://www.linkedin.com/in/hannah-nardone-126655282/',
    'headline': 'Mathematics Student at McMaster University',
    'about': 'Hi! I’m Hannah, I am currently a third year student at McMaster University specializing in mathematics. I am passionate about pure math and eager to pursue research in this field! Hi! I’m Hannah, I am currently a third year student at McMaster University specializing in mathematics. I am passionate about pure math and eager to pursue research in this field!',
    'email': None
}

# Extract dynamic tags and summarize
result = extract_dynamic_tags_and_summarize(input_data)
print(result)

{'tags': "I can analyze the text to extract key tags that best describe the person's background, roles, interests, and areas of expertise. Here are the extracted tags in a Python list of strings:\n\n`['McMaster University', 'Mathematics', 'Student', 'Third year', 'Pure math', 'Research']`\n\nHere's the tag extraction process:\n\n1. The headline mentions the person's affiliation with McMaster University, so we extract `McMaster University` as an institution tag.\n2. The person is a student, so we extract `Mathematics` as a major or area of study tag.\n3. Since the person is in their third year, we extract `Third year` as an academic level tag.\n4. The person is passionate about pure math, so we extract `Pure math` as an interest or area of expertise tag.\n5. The person wants to pursue research in this field, so we extract `Research` as an area of interest or career aspiration tag.\n\nThese tags provide a concise summary of the person's background, roles, interests, and areas of expertis